In [12]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass
import configs.constants as const
import configs.deployments as dep
from azure.identity import ClientSecretCredential

In [3]:
# Define which tools the agent can use to answer user queries
SERP_API_KEY = getpass("Enter your SerpAPI key: ")
search = SerpAPIWrapper(serpapi_api_key=SERP_API_KEY)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [4]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""

In [5]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])# Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [6]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [7]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [8]:
output_parser = CustomOutputParser()

In [14]:
# authenticate to Azure
credentials = ClientSecretCredential(const.IQVIA_TENANT_ID, const.SERVICE_PRINCIPAL, const.SERVICE_PRINCIPAL_SECRET)
token = credentials.get_token(const.SCOPE_NON_INTERACTIVE)
llm = AzureChatOpenAI(
    openai_api_base=f"{const.OPENAI_API_BASE}/{const.OPENAI_API_TYPE}/{const.OPENAI_ACCOUNT_NAME}",
    openai_api_version=const.OPENAI_API_VERSION,
    openai_api_key=token.token,
    deployment_name=dep.GPT_35_TURBO,
    openai_api_type="azure_ad")

In [15]:
# set up the Agent
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [18]:
# use the Agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("Search for Leo DiCaprio's girlfriend on the internet.")



> Entering new  chain...
Thought: I should use a reliable search engine to find the answer.
Action: Search
Action Input: "Leo DiCaprio girlfriend"

Observation:The actor is believed to have recently split from his girlfriend of five years, actor Camila Morrone, but has previously been romantically ...
I need to search for more recent news to see if he has a new girlfriend.
Action: Search
Action Input: "Leo DiCaprio new girlfriend"

Observation:Leo was spotted soaking up the sun on a lit super yacht last Sunday, living his best life with his squad and model Meghan Roche.
I have found the answer to the question.
Final Answer: Leo DiCaprio was recently spotted with model Meghan Roche, suggesting that she may be his current girlfriend.

> Finished chain.


'Leo DiCaprio was recently spotted with model Meghan Roche, suggesting that she may be his current girlfriend.'

In [19]:
# use the Agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("Who is Leo DiCaprio's current girlfriend.")



> Entering new  chain...
Thought: I need to find information on Leo DiCaprio's personal life.
Action: Search
Action Input: "Leo DiCaprio girlfriend"

Observation:The actor is believed to have recently split from his girlfriend of five years, actor Camila Morrone, but has previously been romantically ...
I need to find a more recent source for confirmation.
Action: Search
Action Input: "Leo DiCaprio current girlfriend"

Observation:Just Jared on Instagram: “Leonardo DiCaprio & girlfriend Camila Morrone couple up for a lunch date!
I now know the final answer
Final Answer: Leo DiCaprio's current girlfriend is Camila Morrone.

> Finished chain.


"Leo DiCaprio's current girlfriend is Camila Morrone."

In [20]:
# use the Agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("Who is Leo DiCaprio's current girlfriend? Camila Morrone or Meghan Roche? Give me your evidence and a time clue to the relationship between them.")



> Entering new  chain...
Thought: I need to find evidence of Leo DiCaprio's current girlfriend and the timeline of his relationships with Camila Morrone and Meghan Roche.
Action: Search
Action Input: "Leo DiCaprio girlfriend Camila Morrone Meghan Roche timeline"

Observation:According to Page Six, Roche and DiCaprio are not dating, but she is dating his friend, pop singer Badius. Who is far more age-appropriate for ...
That didn't answer my question, let me try searching again.
Action: Search
Action Input: "Leo DiCaprio relationship timeline Camila Morrone"

Observation:After more than four years of dating, Leonardo DiCaprio and Camila Morrone are going their separate ways. In August, multiple sources told ...
That answers my question. 
Final Answer: Leo DiCaprio's current girlfriend is not Meghan Roche, but rather Camila Morrone. They were in a relationship for over four years but have recently split up.

> Finished chain.


"Leo DiCaprio's current girlfriend is not Meghan Roche, but rather Camila Morrone. They were in a relationship for over four years but have recently split up."

In [16]:
# use the Agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("Who is Leo DiCaprio's current girlfriend? Camila Morrone or Meghan Roche? Give me your evidence and a time clue to the relationship between them. If he doesn't have a girlfriend right now, just say 'he doesn't have a girlfriend right now.'")



> Entering new  chain...
Thought: I need to find information about Leo DiCaprio's current girlfriend to answer this question.
Action: Search
Action Input: "Leo DiCaprio current girlfriend"

Observation:Just Jared on Instagram: “Leonardo DiCaprio & girlfriend Camila Morrone couple up for a lunch date!
Based on the article, it seems that Leo DiCaprio's current girlfriend is Camila Morrone.
Action: Search
Action Input: "Camila Morrone and Meghan Roche"

Observation:This comes as his five-year relationship with Camila Morrone ended in August 2022. Insiders close to Leo have said the pair split after growing ...
Based on the search results, it seems that Leo DiCaprio is not currently dating either Camila Morrone or Meghan Roche, since his relationship with Morrone ended in August 2022.
Thought: I now know the final answer.
Final Answer: Leo DiCaprio doesn't have a girlfriend right now.

> Finished chain.


"Leo DiCaprio doesn't have a girlfriend right now."